<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/99_Utilidades_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
"""
================================================================================
CONVERSOR DE ARCHIVOS RAW NEF A JPEG CON PRESERVACIÓN DE METADATOS EXIF
================================================================================

Sistema de conversión optimizado para archivos RAW .NEF de Nikon a formato JPEG
de alta calidad, preservando metadatos EXIF esenciales para análisis fotográfico.

CONTEXTO:
El objetivo es convertir archivos RAW a un formato más manejable
sin perder información relevante para el análisis de segmentación.

ESTRUCTURA DE DIRECTORIOS:
TFM/0_Imagenes/
├── RAW/                    # Archivos .NEF de entrada
├── EXIF_JSON/             # Metadatos EXIF extraídos (JSON)
└── *.jpg                  # Archivos JPEG de salida

PROBLEMA TÉCNICO RESUELTO:
Los archivos .NEF de Nikon contienen metadatos propietarios (MakerNote) que
exceden el límite de tamaño permitido por la especificación EXIF en archivos
JPEG. Este módulo implementa una estrategia de filtrado selectivo que preserva
únicamente los metadatos relevantes para análisis científico.

METADATOS PRESERVADOS:
- Información de cámara y lente
- Parámetros de exposición (ISO, apertura, velocidad)
- Configuración de captura (modo de exposición, flash, balance de blancos)
- Metadatos temporales y geoespaciales
- Orientación de imagen

METADATOS DESCARTADOS:
- MakerNote propietario de Nikon (datos binarios específicos del fabricante)
- Miniaturas embebidas (thumbnails)
- Información de calibración de sensor
- Datos de procesamiento interno de cámara

CONFIGURACIÓN DE PROCESAMIENTO RAW:
- Algoritmo de demosaicing: AHD (Adaptive Homogeneity-Directed)
- White Balance: Camera WB (preservar intención original del fotógrafo)
- Espacio de color: sRGB (estándar para análisis digital)
- Profundidad de bits: 8 bits por canal (suficiente para análisis de segmentación)
- Gamma: 2.222 (curva sRGB estándar)
- Compresión JPEG: Calidad 98/100, sin submuestreo de croma (4:4:4)

SALIDA:
- Archivos JPEG de alta calidad con metadatos EXIF filtrados
- Archivos JSON con metadatos completos extraídos (backup)
- Informe de conversión en formato JSON

REFERENCIAS:
- Hirakawa, K., & Parks, T. W. (2005). Adaptive homogeneity-directed
  demosaicing algorithm. IEEE Transactions on Image Processing.
- EXIF Specification Version 2.32 (CIPA DC-008-2019)
- LibRaw Documentation: https://www.libraw.org/

Autor: Jesús L.
Proyecto: TFM - Evaluación Comparativa de Técnicas de Segmentación
Universidad: Universidad Oberta de Cataluña (UOC)
Fecha: Noviembre 2025
================================================================================
"""

'\n================================================================================\nCONVERSOR DE ARCHIVOS RAW NEF A JPEG CON PRESERVACIÓN DE METADATOS EXIF\n================================================================================\n\nSistema de conversión optimizado para archivos RAW .NEF de Nikon a formato JPEG\nde alta calidad, preservando metadatos EXIF esenciales para análisis fotográfico.\n\nCONTEXTO:\nEl objetivo es convertir archivos RAW a un formato más manejable\nsin perder información relevante para el análisis de segmentación.\n\nPROBLEMA TÉCNICO RESUELTO:\nLos archivos .NEF de Nikon contienen metadatos propietarios (MakerNote) que\nexceden el límite de tamaño permitido por la especificación EXIF en archivos\nJPEG. Este módulo implementa una estrategia de filtrado selectivo que preserva\núnicamente los metadatos relevantes para análisis científico.\n\nMETADATOS PRESERVADOS:\n- Información de cámara y lente\n- Parámetros de exposición (ISO, apertura, velocidad)\n- Con

In [1]:
!pip install rawpy piexif exifread tqdm

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 kB 2.1 MB/s eta 0:00:00


In [9]:
"""
Conversor NEF → JPG con PRESERVACIÓN COMPLETA de metadatos EXIF
Mantiene TODOS los metadatos de cámara, lente, GPS, etc.
"""

import rawpy
from PIL import Image, ExifTags
import piexif
import exifread
from pathlib import Path
from tqdm import tqdm
import json
from typing import Dict, Any, List, Tuple, Optional
from dataclasses import dataclass, asdict
from datetime import datetime

In [37]:
class GestorOrientacion:
    """
    Gestor de orientación de imágenes basado en metadatos EXIF.

    Esta clase implementa la lectura y corrección de orientación de imágenes
    según la especificación EXIF. La orientación se almacena como un valor
    entero entre 1 y 8 que indica la transformación necesaria.

    VALORES DE ORIENTACIÓN EXIF:
    1 = Normal (0 grados)
    2 = Flip horizontal
    3 = Rotación 180 grados
    4 = Flip vertical
    5 = Flip horizontal + Rotación 270 grados CCW
    6 = Rotación 90 grados CW (270 CCW)
    7 = Flip horizontal + Rotación 90 grados CW
    8 = Rotación 270 grados CW (90 CCW)

    Referencias:
    - EXIF Specification Version 2.32, Section 4.6.4A (Orientation)
    """

    @staticmethod
    def obtener_orientacion(ruta_archivo: Path) -> int:
        """
        Obtiene el valor de orientación EXIF del archivo.

        Args:
            ruta_archivo: Ruta al archivo de imagen

        Returns:
            Valor de orientación EXIF (1-8), 1 por defecto si no existe
        """
        try:
            with open(ruta_archivo, 'rb') as f:
                tags = exifread.process_file(f, details=False)

            if 'Image Orientation' in tags:
                orientacion_str = str(tags['Image Orientation'])

                # Mapeo de strings comunes a valores numéricos
                if 'Horizontal' in orientacion_str and 'normal' in orientacion_str.lower():
                    return 1
                elif 'Rotated 90 CW' in orientacion_str:
                    return 6
                elif 'Right side' in orientacion_str or 'Rotate 90 CW' in orientacion_str:
                    return 6
                elif 'Rotated 180' in orientacion_str or 'Upside-down' in orientacion_str:
                    return 3
                elif 'Rotated 90 CCW' in orientacion_str:
                    return 8
                elif 'Left side' in orientacion_str or 'Rotate 270 CW' in orientacion_str:
                    return 8

                # Intentar convertir directamente a entero
                try:
                    return int(orientacion_str)
                except:
                    pass

            return 1  # Orientación normal por defecto

        except Exception as e:
            print(f"Advertencia: Error leyendo orientación de {ruta_archivo.name}: {e}")
            return 1

    @classmethod
    def corregir_orientacion(cls, imagen: Image.Image, orientacion: int) -> Tuple[Image.Image, int]:
        """
        Corrige la orientación de una imagen según valor EXIF.

        CORRECCIÓN: El RAW procesado con user_flip=0 sale en orientación del sensor.
        Para orientación 8, el sensor captura en vertical (4016x6016) que es correcto.
        NO necesitamos rotar.

        Args:
            imagen: Objeto PIL Image a corregir
            orientacion: Valor de orientación EXIF (1-8)

        Returns:
            Tupla (imagen_corregida, grados_rotacion)
        """
        rotacion_grados = 0

        if orientacion == 1:
            # Normal, no hacer nada
            pass

        elif orientacion == 2:
            # Flip horizontal
            imagen = imagen.transpose(Image.FLIP_LEFT_RIGHT)

        elif orientacion == 3:
            # Rotación 180 grados
            imagen = imagen.transpose(Image.ROTATE_180)
            rotacion_grados = 180

        elif orientacion == 4:
            # Flip vertical
            imagen = imagen.transpose(Image.FLIP_TOP_BOTTOM)

        elif orientacion == 5:
            # Flip horizontal + Rotación 270 CCW
            imagen = imagen.transpose(Image.FLIP_LEFT_RIGHT)
            imagen = imagen.transpose(Image.ROTATE_90)
            rotacion_grados = 270

        elif orientacion == 6:
            # Rotación 90 CW
            imagen = imagen.transpose(Image.ROTATE_270)
            rotacion_grados = 270

        elif orientacion == 7:
            # Flip horizontal + Rotación 90 CW
            imagen = imagen.transpose(Image.FLIP_LEFT_RIGHT)
            imagen = imagen.transpose(Image.ROTATE_270)
            rotacion_grados = 90

        elif orientacion == 8:
            # NO ROTAR - el sensor ya capturó en vertical
            # El RAW con user_flip=0 mantiene la orientación correcta
            pass

        return imagen, rotacion_grados

In [20]:
@dataclass
class ResultadoConversion:
    """
    Estructura de datos para almacenar resultado de conversión individual.

    Attributes:
        archivo_origen: Nombre del archivo NEF de origen
        exito: Indica si la conversión fue exitosa
        exif_copiado: Indica si los metadatos EXIF se copiaron correctamente
        orientacion_corregida: Indica si se aplicó corrección de orientación
        rotacion_aplicada: Grados de rotación aplicados (0, 90, 180, 270)
        tamano_nef_mb: Tamaño del archivo NEF en megabytes
        tamano_jpg_mb: Tamaño del archivo JPG resultante en megabytes
        reduccion_porcentaje: Porcentaje de reducción de tamaño
        tiempo_procesamiento: Tiempo de procesamiento en segundos
        error: Mensaje de error si la conversión falló
    """
    archivo_origen: str
    exito: bool
    exif_copiado: bool
    orientacion_corregida: bool
    rotacion_aplicada: int
    tamano_nef_mb: float
    tamano_jpg_mb: float
    reduccion_porcentaje: float
    tiempo_procesamiento: float
    error: str = None

In [16]:
class ExtractorMetadatosEXIF:
    """
    Extractor de metadatos EXIF relevantes para análisis fotográfico.

    Esta clase implementa la extracción selectiva de metadatos EXIF,
    filtrando únicamente aquellos campos relevantes para el análisis
    de características fotográficas y descartando datos propietarios
    del fabricante que no aportan valor al estudio.
    """

    # Campos EXIF considerados esenciales para análisis fotográfico
    CAMPOS_ESENCIALES = [
        'Image Make',
        'Image Model',
        'Image Orientation',
        'Image DateTime',
        'Image Software',
        'EXIF LensModel',
        'EXIF FocalLength',
        'EXIF FNumber',
        'EXIF ExposureTime',
        'EXIF ISOSpeedRatings',
        'EXIF DateTimeOriginal',
        'EXIF DateTimeDigitized',
        'EXIF ExposureProgram',
        'EXIF MeteringMode',
        'EXIF Flash',
        'EXIF WhiteBalance',
        'EXIF FocalLengthIn35mmFilm',
        'EXIF ExposureMode',
        'EXIF SceneCaptureType',
        'EXIF Contrast',
        'EXIF Saturation',
        'EXIF Sharpness',
        'GPS GPSLatitude',
        'GPS GPSLongitude',
        'GPS GPSAltitude',
    ]

    def extraer_metadatos(self, ruta_archivo: Path) -> Dict[str, str]:
        """
        Extrae metadatos EXIF esenciales de un archivo de imagen.

        Args:
            ruta_archivo: Ruta al archivo de imagen

        Returns:
            Diccionario con metadatos EXIF extraídos
        """
        metadatos = {}

        try:
            with open(ruta_archivo, 'rb') as f:
                tags = exifread.process_file(f, details=False)

            for campo in self.CAMPOS_ESENCIALES:
                if campo in tags:
                    metadatos[campo] = str(tags[campo])

        except Exception as e:
            print(f"Advertencia: Error extrayendo metadatos de {ruta_archivo.name}: {e}")

        return metadatos

In [39]:
class ProcesadorEXIF:
    """
    Procesador de metadatos EXIF para transferencia entre formatos.

    Esta clase implementa la transferencia selectiva de metadatos EXIF
    desde archivos RAW a JPEG, eliminando campos problemáticos que
    exceden las limitaciones del formato JPEG estándar.
    """

    def copiar_exif_filtrado(self, archivo_origen: Path, archivo_destino: Path) -> bool:
        """
        Copia metadatos EXIF filtrando campos problemáticos.

        El MakerNote propietario de Nikon y las miniaturas embebidas
        frecuentemente exceden el límite de tamaño EXIF permitido por
        la especificación JPEG. Este método elimina selectivamente estos
        campos manteniendo todos los metadatos científicamente relevantes.

        Args:
            archivo_origen: Ruta al archivo NEF origen
            archivo_destino: Ruta al archivo JPG destino

        Returns:
            True si la copia fue exitosa, False en caso contrario
        """
        try:
            exif_dict = piexif.load(str(archivo_origen))

            # Eliminar MakerNote propietario
            if "Exif" in exif_dict:
                if piexif.ExifIFD.MakerNote in exif_dict["Exif"]:
                    del exif_dict["Exif"][piexif.ExifIFD.MakerNote]

            # Establecer orientación normal (1) en todos los IFDs
            # La orientación está en ImageIFD, no en ExifIFD
            if "0th" in exif_dict:
                exif_dict["0th"][piexif.ImageIFD.Orientation] = 1

            if "1st" in exif_dict:
                # Eliminar miniaturas embebidas
                if piexif.ImageIFD.JPEGInterchangeFormat in exif_dict["1st"]:
                    del exif_dict["1st"][piexif.ImageIFD.JPEGInterchangeFormat]
                if piexif.ImageIFD.JPEGInterchangeFormatLength in exif_dict["1st"]:
                    del exif_dict["1st"][piexif.ImageIFD.JPEGInterchangeFormatLength]

                # Establecer orientación normal
                exif_dict["1st"][piexif.ImageIFD.Orientation] = 1

            exif_bytes = piexif.dump(exif_dict)

            img = Image.open(archivo_destino)
            img.save(
                archivo_destino,
                'JPEG',
                quality=98,
                exif=exif_bytes,
                optimize=True,
                subsampling=0
            )

            return True

        except Exception as e:
            print(f"Advertencia: Error copiando EXIF a {archivo_destino.name}: {e}")
            return False


class ConversorRAW:
    """
    Conversor de archivos RAW a JPEG con procesamiento científico.

    Esta clase implementa el pipeline completo de conversión desde
    archivos RAW de Nikon (.NEF) a archivos JPEG de alta calidad,
    aplicando procesamiento demosaicing AHD y preservando metadatos
    relevantes para análisis fotográfico.
    """

    def __init__(self):
        """Inicializa el conversor con sus componentes."""
        self.extractor_metadatos = ExtractorMetadatosEXIF()
        self.procesador_exif = ProcesadorEXIF()
        self.gestor_orientacion = GestorOrientacion()

    def procesar_raw(self, ruta_nef: Path) -> Image.Image:
        """
        Procesa archivo RAW aplicando pipeline de conversión científico.

        IMPORTANTE: rawpy.postprocess NO aplica automáticamente la orientación EXIF.
        La imagen sale en orientación del sensor, no en orientación de visualización.

        Args:
            ruta_nef: Ruta al archivo .NEF

        Returns:
            Objeto PIL Image con la imagen procesada
        """
        with rawpy.imread(str(ruta_nef)) as raw:
            rgb = raw.postprocess(
                demosaic_algorithm=rawpy.DemosaicAlgorithm.AHD,
                use_camera_wb=True,
                use_auto_wb=False,
                output_color=rawpy.ColorSpace.sRGB,
                output_bps=8,
                gamma=(2.222, 4.5),
                no_auto_bright=False,
                half_size=False,
                #user_flip=0  # CRÍTICO: 0 = sin rotación automática, procesar como está el sensor
            )

        return Image.fromarray(rgb)

    def convertir_archivo(
        self,
        ruta_nef: Path,
        ruta_jpg: Path,
        ruta_json: Path,
        calidad: int = 98
    ) -> ResultadoConversion:
        """
        Convierte un archivo NEF individual a JPEG con metadatos.
        """
        tiempo_inicio = datetime.now()

        try:
            # Paso 1: Extraer metadatos antes de procesar
            metadatos = self.extractor_metadatos.extraer_metadatos(ruta_nef)

            # Paso 2: Obtener orientación original (solo para logging)
            orientacion = self.gestor_orientacion.obtener_orientacion(ruta_nef)

            # DEPURACIÓN
            print(f"\n{ruta_nef.name}: Orientación EXIF detectada = {orientacion}")

            # Paso 3: Procesar archivo RAW (rawpy aplica orientación automáticamente)
            imagen = self.procesar_raw(ruta_nef)

            print(f"  Tamaño imagen RAW procesada: {imagen.size} (ancho x alto)")

            # Paso 4: NO corregir orientación - rawpy ya la aplicó
            # La imagen ya viene en la orientación correcta

            # Paso 5: Guardar JPEG directamente
            imagen.save(
                ruta_jpg,
                'JPEG',
                quality=calidad,
                optimize=True,
                subsampling=0
            )

            # Paso 6: Copiar metadatos EXIF filtrados
            exif_ok = self.procesador_exif.copiar_exif_filtrado(ruta_nef, ruta_jpg)

            # Paso 7: Guardar backup JSON de metadatos
            metadatos['orientacion_original'] = orientacion
            metadatos['rotacion_aplicada'] = 0  # rawpy ya la aplicó

            with open(ruta_json, 'w', encoding='utf-8') as f:
                json.dump(metadatos, f, indent=2, ensure_ascii=False)

            # Paso 8: Calcular métricas
            tiempo_fin = datetime.now()
            tiempo_procesamiento = (tiempo_fin - tiempo_inicio).total_seconds()

            tamano_nef = ruta_nef.stat().st_size / (1024 * 1024)
            tamano_jpg = ruta_jpg.stat().st_size / (1024 * 1024)
            reduccion = ((tamano_nef - tamano_jpg) / tamano_nef) * 100

            return ResultadoConversion(
                archivo_origen=ruta_nef.name,
                exito=True,
                exif_copiado=exif_ok,
                orientacion_corregida=False,  # rawpy lo hizo automáticamente
                rotacion_aplicada=0,
                tamano_nef_mb=round(tamano_nef, 2),
                tamano_jpg_mb=round(tamano_jpg, 2),
                reduccion_porcentaje=round(reduccion, 1),
                tiempo_procesamiento=round(tiempo_procesamiento, 2)
            )

        except Exception as e:
            return ResultadoConversion(
                archivo_origen=ruta_nef.name,
                exito=False,
                exif_copiado=False,
                orientacion_corregida=False,
                rotacion_aplicada=0,
                tamano_nef_mb=0.0,
                tamano_jpg_mb=0.0,
                reduccion_porcentaje=0.0,
                tiempo_procesamiento=0.0,
                error=str(e)
            )

In [18]:
class SistemaConversionBatch:
    """
    Sistema de conversión por lotes de archivos RAW a JPEG.

    Implementa el procesamiento paralelo de múltiples archivos RAW
    con gestión de errores, reporting y validación de integridad.
    """

    def __init__(self):
        """Inicializa el sistema de conversión por lotes."""
        self.conversor = ConversorRAW()

    def validar_archivo_destino(self, ruta_jpg: Path) -> bool:
        """
        Valida que un archivo JPG no esté corrupto.

        Args:
            ruta_jpg: Ruta al archivo JPG a validar

        Returns:
            True si el archivo es válido, False si está corrupto
        """
        if not ruta_jpg.exists():
            return False

        if ruta_jpg.stat().st_size == 0:
            print(f"Advertencia: Archivo corrupto detectado (0 bytes): {ruta_jpg.name}")
            return False

        return True

    def convertir_directorio(
        self,
        directorio_base: Path,
        calidad: int = 98
    ) -> List[ResultadoConversion]:
        """
        Convierte todos los archivos NEF de un directorio a JPEG.

        ESTRUCTURA DE DIRECTORIOS:
        directorio_base/0_Imagenes/
        ├── RAW/           # Entrada: archivos .NEF
        ├── EXIF_JSON/     # Salida: metadatos JSON
        └── *.jpg          # Salida: archivos JPEG

        Args:
            directorio_base: Directorio base del proyecto (TFM)
            calidad: Calidad de compresión JPEG (1-100)

        Returns:
            Lista de ResultadoConversion con estadísticas de cada archivo
        """
        directorio_base = Path(directorio_base)

        # Definir rutas según estructura especificada
        dir_imagenes = directorio_base / "0_Imagenes"
        dir_raw = dir_imagenes / "RAW"
        dir_json = dir_imagenes / "EXIF_JSON"

        # Crear directorios si no existen
        dir_json.mkdir(parents=True, exist_ok=True)

        # Validar que existe directorio RAW
        if not dir_raw.exists():
            raise FileNotFoundError(f"Directorio RAW no encontrado: {dir_raw}")

        # Escanear archivos NEF
        archivos = (
            list(dir_raw.glob("*.NEF")) +
            list(dir_raw.glob("*.nef"))
        )

        print(f"\n{'='*80}")
        print(f"SISTEMA DE CONVERSIÓN RAW A JPEG")
        print(f"{'='*80}")
        print(f"Archivos detectados:      {len(archivos)}")
        print(f"Calidad JPEG:             {calidad}/100")
        print(f"Algoritmo demosaicing:    AHD (Adaptive Homogeneity-Directed)")
        print(f"Espacio de color:         sRGB")
        print(f"Profundidad bits:         8 bits/canal")
        print(f"Corrección orientación:   Activada")
        print(f"\nESTRUCTURA DE DIRECTORIOS:")
        print(f"  Entrada (NEF):          {dir_raw}")
        print(f"  Salida (JPG):           {dir_imagenes}")
        print(f"  Salida (JSON):          {dir_json}")
        print(f"{'='*80}\n")

        resultados = []

        for archivo_nef in tqdm(archivos, desc="Procesando", unit="archivo"):
            # Definir rutas de salida
            archivo_jpg = dir_imagenes / f"{archivo_nef.stem}.jpg"
            archivo_json = dir_json / f"{archivo_nef.stem}_exif.json"

            # Validar si archivo JPG ya existe y es válido
            if archivo_jpg.exists():
                if self.validar_archivo_destino(archivo_jpg):
                    continue
                else:
                    archivo_jpg.unlink()

            # Convertir archivo
            resultado = self.conversor.convertir_archivo(
                archivo_nef,
                archivo_jpg,
                archivo_json,
                calidad
            )
            resultados.append(resultado)

        # Generar informe
        self._generar_informe(resultados, dir_json)

        return resultados

    def _generar_informe(
        self,
        resultados: List[ResultadoConversion],
        directorio_json: Path
    ) -> None:
        """
        Genera informe consolidado de conversión.

        Args:
            resultados: Lista de resultados de conversión
            directorio_json: Directorio donde guardar informe
        """
        exitosos = sum(1 for r in resultados if r.exito)
        con_exif = sum(1 for r in resultados if r.exif_copiado)
        con_rotacion = sum(1 for r in resultados if r.orientacion_corregida)
        fallidos = len(resultados) - exitosos

        if resultados:
            avg_nef = sum(r.tamano_nef_mb for r in resultados) / len(resultados)
            avg_jpg = sum(r.tamano_jpg_mb for r in resultados) / len(resultados)
            avg_tiempo = sum(r.tiempo_procesamiento for r in resultados) / len(resultados)
            avg_reduccion = sum(r.reduccion_porcentaje for r in resultados) / len(resultados)
        else:
            avg_nef = avg_jpg = avg_tiempo = avg_reduccion = 0.0

        print(f"\n{'='*80}")
        print(f"RESUMEN DE CONVERSIÓN")
        print(f"{'='*80}")
        print(f"Total procesados:              {len(resultados)}")
        print(f"Conversiones exitosas:         {exitosos}")
        print(f"Con metadatos EXIF:            {con_exif}")
        print(f"Con corrección orientación:    {con_rotacion}")
        print(f"Conversiones fallidas:         {fallidos}")
        print(f"\nESTADÍSTICAS DE TAMAÑO:")
        print(f"Tamaño promedio NEF:           {avg_nef:.2f} MB")
        print(f"Tamaño promedio JPG:           {avg_jpg:.2f} MB")
        print(f"Reducción promedio:            {avg_reduccion:.1f}%")
        print(f"\nRENDIMIENTO:")
        print(f"Tiempo promedio/archivo:       {avg_tiempo:.2f} segundos")
        print(f"{'='*80}\n")

        informe = {
            'fecha_generacion': datetime.now().isoformat(),
            'estadisticas_agregadas': {
                'total_archivos': len(resultados),
                'exitosos': exitosos,
                'con_exif': con_exif,
                'con_correccion_orientacion': con_rotacion,
                'fallidos': fallidos,
                'tamano_promedio_nef_mb': round(avg_nef, 2),
                'tamano_promedio_jpg_mb': round(avg_jpg, 2),
                'reduccion_promedio_porcentaje': round(avg_reduccion, 1),
                'tiempo_promedio_segundos': round(avg_tiempo, 2)
            },
            'resultados_individuales': [asdict(r) for r in resultados]
        }

        ruta_informe = directorio_json / "informe_conversion.json"
        with open(ruta_informe, 'w', encoding='utf-8') as f:
            json.dump(informe, f, indent=2, ensure_ascii=False)

        print(f"Informe detallado guardado:    {ruta_informe}")
        print(f"Metadatos EXIF guardados en:   {directorio_json}\n")


In [19]:
def main():
    """
    Función principal de ejecución del sistema de conversión.

    Estructura de directorios requerida:
    /content/drive/MyDrive/TFM/0_Imagenes/RAW/  <- Archivos .NEF aquí
    """
    sistema = SistemaConversionBatch()

    resultados = sistema.convertir_directorio(
        directorio_base="/content/drive/MyDrive/TFM",
        calidad=98
    )

    return resultados

In [40]:
if __name__ == "__main__":
    resultados = main()


SISTEMA DE CONVERSIÓN RAW A JPEG
Archivos detectados:      19
Calidad JPEG:             98/100
Algoritmo demosaicing:    AHD (Adaptive Homogeneity-Directed)
Espacio de color:         sRGB
Profundidad bits:         8 bits/canal
Corrección orientación:   Activada

ESTRUCTURA DE DIRECTORIOS:
  Entrada (NEF):          /content/drive/MyDrive/TFM/0_Imagenes/RAW
  Salida (JPG):           /content/drive/MyDrive/TFM/0_Imagenes
  Salida (JSON):          /content/drive/MyDrive/TFM/0_Imagenes/EXIF_JSON



Procesando:   0%|          | 0/19 [00:00<?, ?archivo/s]


_DSC0084.NEF: Orientación EXIF detectada = 8
  Tamaño imagen RAW procesada: (4016, 6016) (ancho x alto)


Procesando:   5%|▌         | 1/19 [00:09<02:53,  9.67s/archivo]


_DSC0411.NEF: Orientación EXIF detectada = 8
  Tamaño imagen RAW procesada: (4016, 6016) (ancho x alto)


...


_DSC0283.NEF: Orientación EXIF detectada = 8
  Tamaño imagen RAW procesada: (4016, 6016) (ancho x alto)


Procesando:  95%|█████████▍| 18/19 [02:21<00:07,  7.52s/archivo]


_DSC0449.NEF: Orientación EXIF detectada = 8
  Tamaño imagen RAW procesada: (4016, 6016) (ancho x alto)


Procesando: 100%|██████████| 19/19 [02:29<00:00,  7.89s/archivo]


RESUMEN DE CONVERSIÓN
Total procesados:              19
Conversiones exitosas:         19
Con metadatos EXIF:            19
Con corrección orientación:    0
Conversiones fallidas:         0

ESTADÍSTICAS DE TAMAÑO:
Tamaño promedio NEF:           17.12 MB
Tamaño promedio JPG:           11.18 MB
Reducción promedio:            35.0%

RENDIMIENTO:
Tiempo promedio/archivo:       7.88 segundos

Informe detallado guardado:    /content/drive/MyDrive/TFM/0_Imagenes/EXIF_JSON/informe_conversion.json
Metadatos EXIF guardados en:   /content/drive/MyDrive/TFM/0_Imagenes/EXIF_JSON

